# 

In [1]:
import numpy as np
import copy
import torch
from transformers import logging
logging.set_verbosity_error()

from PIL import Image
import matplotlib.pyplot as plt

from fromage import models
from fromage import utils

In [2]:
# Load model used in the paper.
model_dir = '/home/ubuntu/Project/Project_final/11-777-Project/fromage/runs/frozen_15'
model = models.load_fromage(model_dir,type1='ss')

Using HuggingFace AutoFeatureExtractor for openai/clip-vit-large-patch14.
Using facebook/opt-1.3b for the language model.
Using openai/clip-vit-large-patch14 for the visual model with 4 visual tokens.


/home/ubuntu/anaconda3/envs/lavis/lib/python3.8/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


Freezing the LM.


/home/ubuntu/anaconda3/envs/lavis/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [117]:
import json
dataset_test=json.load(open("/home/ubuntu/Project/11-777-Project-aa/Data/Experiments/Config 2/retrieval_4.json","r"))

In [118]:
import pickle

# Specify the path to your pickle file
pickle_file_path = '/home/ubuntu/Project/11-777-Project-aa/Data/Experiments/Config 2/img_embedding.pkl'

# Load the data from the pickle file
with open(pickle_file_path, 'rb') as file:
    loaded_data = pickle.load(file)

# Now, 'loaded_data' contains the data stored in the pickle file


In [119]:
import pickle
count=0
for i1,j1 in zip(loaded_data[0],loaded_data[1]):
    pickle_file_path = f'/home/ubuntu/Project/11-777-Project-aa/Data/uni_test/{i1}.pkl'
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(j1.reshape(1,-1), file)

In [120]:
import json

def load_file(path, limit=50):
    result = {}
    with open(path, 'r') as file:
        for index, line in enumerate(file):
            if index >= limit:
                break  # Stop reading after the specified limit
            dico = json.loads(line)
            result[dico["qid"]] = dico
    return result

path_test = "/home/ubuntu/Project/11-777-Project-aa/Data/Experiments/test.json"
test_data = load_file(path_test, limit=50)


In [121]:
import pickle
import os
def generate_answer_img(Q,c1,c2):
    if c2==None:
        gold_feature = []
        image_feature_path1='/home/ubuntu/Project/11-777-Project-aa/Data/uni_test/'+str(c1)+'.pkl'
        image_feature_path2='/home/ubuntu/Project/11-777-Project-aa/Data/uni_test/'+str(c1)+'.pkl'
        gold_feature.append(image_feature_path1)
        gold_feature.append(image_feature_path2)
        c=[gold_feature]
        prompt1= 'Q: '+  Q+ '\nA:'
        A1=model.generate_for_images_and_texts([c,prompt1], num_words=40)
        # prompt2= 'Q: '+  q2+ '\nA:'
        # A2=model.generate_for_images_and_texts([c,prompt2], num_words=40)
        # prompt3= 'Q: '+  q1+ ' \nA: ' +A1[0] + '\nQ: '+  q2+ ' \nA: ' +A2[0] + '\nQ: '+  Q+ ' \nA:'
        # A_final=model.generate_for_images_and_texts([prompt3], num_words=40)
        # A_final=A1
    else:
        gold_feature = []
        image_feature_path1='/home/ubuntu/Project/11-777-Project-aa/Data/uni_test/'+str(c1)+'.pkl'
        image_feature_path2='/home/ubuntu/Project/11-777-Project-aa/Data/uni_test/'+str(c2)+'.pkl'
        gold_feature.append(image_feature_path1)
        gold_feature.append(image_feature_path2)
        c=[gold_feature]
        # prompt1= 'Q: '+  q1+ '\nA:'
        # A1=model.generate_for_images_and_texts([c,prompt1], num_words=40)
        # prompt2= 'Q: '+  q2+ '\nA:'
        # A2=model.generate_for_images_and_texts([c,prompt2], num_words=40)
        # prompt3= 'Q: '+  q1+ ' \nA: ' +A1[0] + '\nQ: '+  q2+ ' \nA: ' +A2[0] + '\nQ: '+  Q+ ' \nA:'
        # A_final=model.generate_for_images_and_texts([prompt3], num_words=40)
        prompt1= 'Q: '+  Q+ '\nA:'
        A1=model.generate_for_images_and_texts([c,prompt1], num_words=40)
    return A1[0]

In [110]:
for i in dataset_test:
    print(dataset_test[i]['pos'])
    break
    

['30079629']


In [122]:
Answers_real_im1 = []
Answers_pred_im1 = []
for i in test_data:
    data = test_data[i]
    Q=data['Q']
    A=data['A']
    if(len(dataset_test[i]['pos']))==1:
        c1=dataset_test[i]['pos'][0]
        if(c1[0]=='d'):
            continue
        # print(c1)
        c2=None
        Answers_real_im1.append(A)
        Answers_pred_im1.append(generate_answer_img(Q,c1,c2))

In [123]:
Answers_real_im = []
Answers_pred_im = []
for i in test_data:
    data = test_data[i]
    Q=data['Q']
    A=data['A']
    if(len(dataset_test[i]['pos']))>1:
        c1=dataset_test[i]['pos'][0]
        c2=dataset_test[i]['pos'][1]
        Answers_real_im.append(A)
        Answers_pred_im.append(generate_answer_img(Q,c1,c2))

In [124]:
len(Answers_real_im1), len(Answers_pred_im1)

(33, 33)

In [125]:
Answers_real=Answers_real_im1+Answers_real_im
Answers_pred=Answers_pred_im1+Answers_pred_im

In [126]:
# save as a pkl file
import pickle
with open('Answers_real_manh_2.pkl', 'wb') as f:
    pickle.dump(Answers_real, f)

In [127]:
import pickle
with open('Answers_pred_manh_2.pkl', 'wb') as f:
    pickle.dump(Answers_pred, f)

In [136]:
import pickle
with open('/home/ubuntu/Project/Project_final/11-777-Project/fromage/Answers_pred_manh_3.pkl', 'rb') as f:
    answers_pred_1 = pickle.load(f)
import pickle
with open('/home/ubuntu/Project/Project_final/11-777-Project/fromage/Answers_real_manh_3.pkl', 'rb') as f:
    answers_real_1 = pickle.load(f)

In [137]:
for i,j in zip(answers_real_1,answers_pred_1):
    print('real answer +++++++++++++++++++++++++++++++++')
    print(i[0][1:-1])
    print('pred answer +++++++++++++++++++++++++++++++++')
    print(j)

real answer +++++++++++++++++++++++++++++++++
No, a Minnetonka Rhododendron flower does not have petals in a cup shape.
pred answer +++++++++++++++++++++++++++++++++
 Yes, a Minnetonka Rhododendron flower has petals in a cup shape.
real answer +++++++++++++++++++++++++++++++++
A fountain is sitting in front of the Torre del Reloj.
pred answer +++++++++++++++++++++++++++++++++
 A water-related object.
real answer +++++++++++++++++++++++++++++++++
Blue footprints are painted down the center of Salou Boulevard.
pred answer +++++++++++++++++++++++++++++++++
 The shapes are painted down the center of Salou Boulevard.
real answer +++++++++++++++++++++++++++++++++
Yes, there are sidewalks on both sides of the Mid-Hudson Bridge.
pred answer +++++++++++++++++++++++++++++++++
 Yes, there are sidewalks on both sides of the Mid-Hudson Bridge.
real answer +++++++++++++++++++++++++++++++++
There were red flowers outside the hotel.
pred answer +++++++++++++++++++++++++++++++++
 Yellow
real answer +++